#Modelos de redes secuenciales

##Activación para Colab

In [0]:
#LMG: Variable para controlar si estamos en Colab o no y aplicar a las celdas:
inColab = True

En la siguiente celda, montamos la carpeta personal del Drive en Colab en caso de estar en este entorno. Pedirá un código al que se accede desde el enlace que facilita.

In [0]:
#LMG: Para Google CoLab tener el repo de Drive:
if inColab:
  from google.colab import drive
  drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


##Carga de librerías y preparación de datos

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM
from sklearn.preprocessing import StandardScaler
from keras.optimizers import SGD

In [0]:
#LMG: Para CoLab en sample_data:
if inColab:
  data_consumo = pd.read_csv("/content/drive/My Drive/TFM/data/data_total.csv")
else:
  data_consumo = pd.read_csv("data_total.csv")

x_data = data_consumo.drop(['PVPC_DEF','PVPC_2_PED_NOC','PVPC_ELEC_NOC'], axis=1).copy()
# Delete unconsistent columns
x_data = x_data.drop(columns=['fecha','fecha_dia'])

#y_data = data_consumo[['PVPC_DEF','PVPC_2_PED_NOC','PVPC_ELEC_NOC']]
y_data = data_consumo[['PVPC_DEF']]

x_data.head()

,Unnamed: 0,Demanda,Eolica,Nuclear,Solar,Solar_Fotovoltaica,Solar_Termica,Termica_Renovable,Prevista,Programada,Holiday
0,0,24984.666667,1003.666667,6012.833333,402.166667,33.0,368.500000,580.166667,24691.833333,24517.0,0.0
1,1,23550.833333,899.000000,6013.166667,174.833333,33.0,141.333333,584.166667,23440.000000,23169.0,0.0
2,2,22648.166667,927.500000,6013.500000,103.833333,33.0,70.666667,585.833333,22521.166667,22437.0,0.0
3,3,22203.833333,935.666667,6013.000000,102.000000,33.0,69.000000,588.500000,22335.333333,22281.0,0.0
4,4,21987.833333,900.833333,6014.166667,101.666667,33.0,68.166667,589.000000,22177.833333,21910.0,0.0


In [0]:
# Split the data
x_train, x_valid, y_train, y_valid = train_test_split(x_data, y_data, test_size=0.33, shuffle=True)

#Reshape for the LSTM
x_train = x_train.to_numpy()
x_valid = x_valid.to_numpy()
y_train = y_train.to_numpy()
y_valid = y_valid.to_numpy()

print('Xtrain_dim:', x_train.shape)
print('Ytrain_dim:', y_train.shape)

Xtrain_dim: (22510, 11)
Ytrain_dim: (22510, 1)


##Ejemplo de LSTM

Basado en el ejemplo completo que está explicado en https://adventuresinmachinelearning.com/keras-lstm-tutorial/

Definamos los tamaños de nuestra red y algunas variables adicionales:

En nuestro caso, la entrada va a ser de *1* **x** *nº de datos tomados* **x** *nº de variables independientes*, siendo:

- batch_size, para cada instante tomamos los datos en paquetes.
- nº de datos tomados o num_steps, esto es, en el análisis secuencial cada registro energético completo con el que contamos, o en términos más coloquiales, cuántas filas del dataset se tienen en cuenta .
- nº de variables independientes o num_var, cada registro en los datos de entrada, o en términos más coloquiales, cada columna del dataset.

Además:

- hidden_size, número de unidades en cada célula del LSTM.

- A la salida, tenemos los tres valores a estimar.

In [0]:
batch_size = 16
num_steps = 25
num_var = x_train.shape[1]
hidden_size = 500
output_size = y_train.shape[1]

Para que se ejecute en paquetes, replicamos el dataset de n en n lotes:

In [0]:
# split a sequence into samples
def split_sequence(sequence, n_steps):
  X = list()
  for i in range(len(sequence)):
    # find the end of this pattern
    end_ix = i + n_steps
    # check if we are beyond the sequence
    if end_ix > len(sequence)-1:
      break
    # gather input and output parts of the pattern
    seq_x = sequence[i:end_ix,:]
    X.append(seq_x)

  return np.array(X)

Después, estandarizamos los datos y aplicamos la función a la x e y, agrupando las salidas por paquetes de num_steps:

In [0]:
scaler = StandardScaler()
x_train_est = scaler.fit_transform(x_train)
y_train_est = scaler.fit_transform(y_train)
x_valid_est = scaler.fit_transform(x_valid)
y_valid_est = scaler.fit_transform(y_valid)

x_train_data = split_sequence(x_train_est, num_steps)
y_train_data = y_train_est[num_steps:,:]
x_valid_data = split_sequence(x_valid_est, num_steps)
y_valid_data = y_valid_est[num_steps:,:]

print('Entrenamiento:',x_train_data.shape, y_train_data.shape)
print('Test:',x_valid_data.shape, y_valid_data.shape)

Entrenamiento: (22485, 25, 11) (22485, 1)
Test: (11063, 25, 11) (11063, 1)


A la capa LSTM se le pasa cada vez un instante, siendo el primero el t1, el segundo el t2, etc.

Respecto al ejemplo en la página, hemos:
- Quitado el embedding inicial, ya que no necesitamos codificar la entrada (ya son valores en sí mismos).
- Cambiado el hidden-layer-size. En la página dicen que se suele poner al tamaño de entrada de cada registro. Viene a ser el símil de unidades en una capa densa.
- Dejado una única capa LSTM.
- Eliminado el dropout.
- TimeDistributed usa una capa densa para cada step del entrenamiento. La quitamos también y dejamos la densa exclusivamente, ya que hace la salida muy grande.


In [0]:
# define model
model = Sequential()
model.add(LSTM(hidden_size, activation='relu', return_sequences=True, input_shape=(num_steps,num_var),kernel_initializer='zeros'))
model.add(LSTM(hidden_size, activation='relu'))
model.add(Dense(output_size, activation='linear'))

# compile mode
opt = SGD(lr=0.01, momentum=0.9, decay=1e-6, nesterov=True)
model.compile(optimizer=opt, loss='mse',metrics=['mse'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_38 (LSTM)               (None, 25, 500)           1024000   
_________________________________________________________________
lstm_39 (LSTM)               (None, 500)               2002000   
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 501       
Total params: 3,026,501
Trainable params: 3,026,501
Non-trainable params: 0
_________________________________________________________________
None


Y entrenamos:

In [0]:
history_train = model.fit(x_train_data, y_train_data, epochs=10, batch_size=batch_size, verbose=1)

# summarize history for loss
plt.plot(history_train.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

Epoch 1/10
  160/22485 [..............................] - ETA: 29:15 - loss: 1.1366 - mean_squared_error: 1.1366

Para el test:

In [0]:
scores_test = model.evaluate(x_valid_data, y_valid_data, batch_size=batch_size, verbose=1)

# summarize loss
print("%s: %.2f" % (model.metrics_names[1], scores_test[1]))

11063/11063 [==============================] - 2s 185us/step
mean_squared_error: 99.98%
